In [1]:
import os
import json
import os
import subprocess
import platform

if "program" in os.getcwd():
    os.chdir("..")

experiment_name = 'analysis_baseline' #'cnn_dropout' #'MNIST_FashionMNIST_reg_comparison_again_KDE_adjusted'#'CIFAR10_SVHN_reg_comparison_again_KDE_adjusted' #

In [2]:
os.getcwd()

'/Users/antoniosquicciarini/PhD_projects/memorization_under_label_noise'

In [5]:
path = "models/baseline_models"
folders = os.listdir(path)


In [6]:
sims = []


for folder in folders:
    if os.path.exists(f"{path}/{folder}") and os.path.isfile(f"{path}/{folder}/args.json"):
        with open(f"{path}/{folder}/args.json", 'r') as file:
            content = file.read()
            print(f"Content of {folder}/args.json:\n{content}")
            try:
                sims.append(json.loads(content))
                print(f"{folder} UPLOADED")
            except json.JSONDecodeError as e:
                print(f"Error in {folder}/args.json: {e}")
    else:
        print(f"{folder}/args.json does not exist")

Content of exp_2024-12-20_13-40_MNIST_FashMNIST_baseline_sim_1/args.json:
{
    "verbose": false,
    "verbose_1": false,
    "dataset": "FashionMNIST",
    "network": "Preact_Base_FCN",
    "experiment": "baseline_models",
    "n_layers": 5,
    "n_neurons_x_layer": 500,
    "batch_normalization_flag": false,
    "dropout_rate": 0.0,
    "noise_type": "label_noise",
    "symmetric_label_noise_ratio": 0.4,
    "batch_size": 512,
    "fixed_batch_size": 512,
    "epochs": 100,
    "optimizer": "SGD",
    "loss_function": "cross_entropy_loss",
    "metrics": [
        "CND"
    ],
    "PMFs_epochs_to_plot": [
        0,
        5,
        10,
        20,
        30,
        40,
        50,
        60,
        70,
        80,
        90,
        99
    ],
    "CND_type": [
        "CND_PMF",
        "CND_GL",
        "CND_KDE"
    ],
    "neuron_indexes": [],
    "layer_indexes": [],
    "CND_min_th": 0.0,
    "CND_min_ist": 0,
    "JSD_bins": 40,
    "lr": 0.1,
    "momentum": 0.9,
    "

In [12]:

#filters = [('network', 'Base_FCN'), ('n_layers', 7), ('dropout_rate', 0.0), ('noise_type', 'none')]
#filters = [('network', 'Base_FCN'), ('dropout_rate', 0.0), ('noise_type', 'label_noise')]
#filters = [('network', 'VGG19'), ("dataset", "CIFAR10"), ('noise_type', 'none')]
#filters = [('network', 'ResNet9'), ("dataset", "CIFAR10"), ('JSD_bins', 40)]
#filters = [('network', 'Preact_Base_FCN'), ("dataset", "MNIST")]#, ('JSD_bins', 40)]
#filters = [('CND_well_classified_filter', False)]
#filters = [('network', 'ResNet18'), ('noise_type', 'label_noise')]
#filters = [('network', 'Preact_Base_FCN'), ('dataset', 'FashionMNIST'), ('n_layers', 7)]
#filters = [('dataset', 'FashionMNIST'), ('epochs', 80)]

from datetime import datetime

def filter_sims_by_date(sims, filters, date_str=None, before=True, required_attribute=None):
    """
    Filter simulations based on attribute filters, date, required attribute presence,
    and whether the folder contains a performances.pkl file.

    Args:
        sims (list): List of simulations.
        filters (list): List of (key, value) tuples for filtering.
        date_str (str): Date string in "YYYY-MM-DD" format.
        before (bool): Whether to filter simulations before the date.
        required_attribute (str): Attribute that must be present in the simulation.

    Returns:
        list: Filtered simulations.
    """
    # Convert date_str to a datetime object only if it's provided
    target_date = datetime.strptime(date_str, "%Y-%m-%d") if date_str else None

    # List to store filtered simulations
    sims_filt = []

    for sim in sims:
        # Check for the presence of the required attribute
        if required_attribute and required_attribute not in sim:
            continue

        # Apply attribute-based filtering
        if all(sim.get(key) == value for key, value in filters):
            # Check if the performances.pkl file exists
            results_dir = sim.get("results_dir", "")
            performances_file = os.path.join(results_dir, "performances.pkl")
            if not os.path.isfile(performances_file):
                print(f"Skipping {results_dir}: performances.pkl not found.")
                continue

            # If no date is provided, just append the simulation based on attribute filters
            if not target_date:
                sims_filt.append(sim)
                print(f"FOUND {sim['results_dir']}")
            else:
                # Extract the date from 'results_dir' if filtering by date
                try:
                    # Get date portion using pattern (exp_YYYY-MM-DD)
                    sim_date_str = sim['results_dir'].split("exp_")[1].split("_")[0]
                    sim_date = datetime.strptime(sim_date_str, "%Y-%m-%d")

                    # Filter by date based on 'before' or 'after'
                    if (before and sim_date < target_date) or (not before and sim_date > target_date):
                        sims_filt.append(sim)
                        print(f"FOUND {sim['results_dir']}")
                except (IndexError, ValueError) as e:
                    print(f"Error parsing date for simulation {sim['results_dir']}: {e}")

    return sims_filt

# Example usage
#filters = [('network', 'ResNet9'), ('epochs', 80), ('batch_size', 256), ('dataset', 'SVHN')]
#filters = [('network', 'ResNet9'), ('epochs', 120), ('batch_size', 256), ('dataset', 'CIFAR10')]
#filters = [('network', 'Preact_Base_FCN'), ('epochs', 100), ('batch_size', 512), ("lr_milestones", [60, 80, 90])]#, ('CND_reg_type', 'KDE_FFT')]
#filters = [('experiment', experiment_name)]#,('network', 'Preact_Base_FCN')]
#filters = [('experiment', experiment_name)]#,('network', 'Preact_Base_FCN')]
filters = [('dataset', 'MNIST')]

required_attribute = 'CND_reg_type'
date_str = "2024-11-01"
filtered_sims = filter_sims_by_date(sims, filters, date_str=date_str, before=False)#, required_attribute=required_attribute)


Skipping models/baseline_models/exp_2024-12-20_14-18_MNIST_FashMNIST_baseline_sim_0: performances.pkl not found.
FOUND models/baseline_models/exp_2024-12-20_14-31_MNIST_FashMNIST_baseline_sim_0
Skipping models/baseline_models/exp_2024-12-20_13-54_MNIST_FashMNIST_baseline_sim_0: performances.pkl not found.
Skipping models/baseline_models/exp_2024-12-20_13-41_MNIST_FashMNIST_baseline_sim_0: performances.pkl not found.
Skipping models/baseline_models/exp_2024-12-20_13-52_MNIST_FashMNIST_baseline_sim_0: performances.pkl not found.
Skipping models/baseline_models/exp_2024-12-20_13-57_MNIST_FashMNIST_baseline_sim_0: performances.pkl not found.
Skipping models/baseline_models/exp_2024-12-20_13-48_MNIST_FashMNIST_baseline_sim_0: performances.pkl not found.
FOUND models/baseline_models/exp_2024-12-20_12-37_MNIST_FashMNIST_baseline_sim_0
Skipping models/baseline_models/exp_2024-12-20_13-45_MNIST_FashMNIST_baseline_sim_0: performances.pkl not found.
FOUND models/baseline_models/exp_2024-12-19_15-

In [10]:
[sim['filename'] for sim in filtered_sims]


[]

In [10]:
from PIL import Image
import os
import re  # Regular expression module

def combine_images_with_fixed_and_pattern(folder_path, fixed_images, pattern, output_path="combined_image.jpg"):
    """
    Combine fixed images and multiple images matching a pattern into a single vertical column and save the result.

    Args:
        folder_path (str): Path to the folder containing images.
        fixed_images (list): List of fixed image file names to include.
        pattern (str): Regex pattern to match additional images.
        output_path (str): Path to save the combined image.
    """
    # Collect fixed images
    images = []
    for file_name in fixed_images:
        file_path = os.path.join(folder_path, file_name)
        if os.path.exists(file_path):
            images.append(Image.open(file_path))
        else:
            print(f"Fixed image not available: {file_path}")

    # Find all files in the folder matching the pattern
    for file_name in os.listdir(folder_path):
        if re.match(pattern, file_name):  # Match files based on the provided pattern
            file_path = os.path.join(folder_path, file_name)
            if os.path.exists(file_path):
                images.append(Image.open(file_path))
            else:
                print(f"Pattern-matched image not available: {file_path}")

    # If no images are available, exit
    if not images:
        print("No images to combine.")
        return

    # Assume all images are the same size
    width, height = images[0].size

    # Calculate dimensions for the combined image (mono column)
    combined_width = width
    combined_height = len(images) * height

    # Create a blank image with the combined dimensions
    combined_image = Image.new('RGB', (combined_width, combined_height))

    # Paste each image into the combined image
    for index, image in enumerate(images):
        y = index * height
        combined_image.paste(image, (0, y))

    # Save the combined image
    combined_image.save(output_path)
    print(f"Combined image saved as {output_path}")

# Usage
for sim in filtered_sims:
    folder_path = f"/Volumes/T7/PhD_projects/novel_entropic_divergences_DNN_optimization/{sim['results_dir']}"
    
    # Fixed images to include
    fixed_images = [
        f"training_curves_{sim['timestamp']}.png",
        f"PC_{sim['timestamp']}.png"
    ]
    
    # Pattern to match additional images
    pattern = r"CND_.*_CND_mean_and_std_.*\.png"
    
    # Output file path
    output_path = os.path.join(folder_path, f"{sim['filename']}_combined_image.png")
    
    # Check if the combined image already exists
    if not os.path.exists(output_path):
        combine_images_with_fixed_and_pattern(folder_path, fixed_images, pattern, output_path=output_path)
    else:
        print(f"Combined image already exists: {output_path}")

Combined image already exists: /Volumes/T7/PhD_projects/novel_entropic_divergences_DNN_optimization/results/memorization_images/exp_2024-12-04_01-25_MNIST_FashMNIST_baseline_sim_9/exp_2024-12-04_01-25_MNIST_FashMNIST_baseline_sim_9_combined_image.png
Combined image already exists: /Volumes/T7/PhD_projects/novel_entropic_divergences_DNN_optimization/results/memorization_images/exp_2024-12-04_00-37_MNIST_FashMNIST_baseline_sim_8/exp_2024-12-04_00-37_MNIST_FashMNIST_baseline_sim_8_combined_image.png
Combined image already exists: /Volumes/T7/PhD_projects/novel_entropic_divergences_DNN_optimization/results/memorization_images/exp_2024-12-03_22-37_MNIST_FashMNIST_baseline_sim_5/exp_2024-12-03_22-37_MNIST_FashMNIST_baseline_sim_5_combined_image.png
Combined image already exists: /Volumes/T7/PhD_projects/novel_entropic_divergences_DNN_optimization/results/memorization_images/exp_2024-12-03_20-19_MNIST_FashMNIST_baseline_sim_2/exp_2024-12-03_20-19_MNIST_FashMNIST_baseline_sim_2_combined_image.

In [11]:
#OPEN FOLDERS
filtered_sims[0]['results_dir']

def open_folder(path):
    if platform.system() == "Windows":
        os.startfile(path)
    elif platform.system() == "Darwin":  # macOS
        subprocess.Popen(["open", path])
    else:  # Linux
        subprocess.Popen(["xdg-open", path])
len(filtered_sims)
# Use your folder path

# for sim in filtered_sims:
#     open_folder(f"/Volumes/T7/PhD_projects/novel_entropic_divergences_DNN_optimization/{sim['results_dir']}")


27

In [12]:
# LOAD PERFORMANCES.PKL FILE
import pickle

performances_list = []
for sim in filtered_sims:
    pickle_file_path = f"/Volumes/T7/PhD_projects/novel_entropic_divergences_DNN_optimization/{sim['results_dir']}/performances.pkl"
    # Load the pickle file
    try:
        with open(pickle_file_path, "rb") as file:
            performances_list.append(pickle.load(file))
        print(f"{sim['filename']} - Pickle file loaded successfully!")
    except FileNotFoundError:
        print(f"File not found: {pickle_file_path}")
    except pickle.UnpicklingError as e:
        print(f"Error loading pickle file: {e}")


exp_2024-12-04_01-25_MNIST_FashMNIST_baseline_sim_9 - Pickle file loaded successfully!
exp_2024-12-04_00-37_MNIST_FashMNIST_baseline_sim_8 - Pickle file loaded successfully!
exp_2024-12-03_22-37_MNIST_FashMNIST_baseline_sim_5 - Pickle file loaded successfully!
exp_2024-12-03_20-19_MNIST_FashMNIST_baseline_sim_2 - Pickle file loaded successfully!
exp_2024-12-04_11-33_SVHN_baseline_sim_0 - Pickle file loaded successfully!
exp_2024-12-04_04-16_MNIST_FashMNIST_baseline_sim_13 - Pickle file loaded successfully!
exp_2024-12-04_02-51_MNIST_FashMNIST_baseline_sim_11 - Pickle file loaded successfully!
exp_2024-12-03_19-32_MNIST_FashMNIST_baseline_sim_1 - Pickle file loaded successfully!
exp_2024-12-09_17-58_CIFAR10_baseline_sim_2 - Pickle file loaded successfully!
exp_2024-12-04_09-45_CIFAR10_baseline_sim_2 - Pickle file loaded successfully!
exp_2024-12-04_05-37_MNIST_FashMNIST_baseline_sim_15 - Pickle file loaded successfully!
exp_2024-12-03_21-45_MNIST_FashMNIST_baseline_sim_4 - Pickle file l

/Users/antoniosquicciarini/miniconda/envs/env_pytorch_new/lib/python3.9/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(i

In [13]:
for ii, sim in enumerate(filtered_sims):
    if 'CND_reg_gamma' not in sim:
        sim['CND_reg_gamma'] = 0
        sim['CND_reg_type'] = 'none'
        sim['CND_reg_only_last_layer'] = False
    
    if sim['noise_type'] == "worse":
        sim['noise_type'] = "none"


In [14]:
import os
import pandas as pd
import ace_tools_open as ace  # Replace `ace_tools` with `ace_tools_open` or `ace`

# Create an empty list to collect data for the DataFrame
data = []

# Directory to save the results
results_dir = "/Volumes/T7/PhD_projects/novel_entropic_divergences_DNN_optimization/results/memorization_reg"

# Ensure the directory exists
os.makedirs(results_dir, exist_ok=True)

# Define filters for noise_types and CND_reg
selected_noise_types = ["label_noise", "none"]  # Add the noise types you want to include
selected_CND_regs = ["GL", "mean", "none", "KDE_FFT"]          # Add the CND_reg types you want to include

# Iterate through performances and simulations to extract relevant data
for ii, performances in enumerate(performances_list):
    try:
        # Extract relevant information
        name = filtered_sims[ii]["filename"]
        noise_type = filtered_sims[ii]['noise_type']
        dataset = filtered_sims[ii]["dataset"]
        test_acc = performances.test_acc[-1].item()  # Assuming 'test_acc' is stored in performances
        misslab_acc = performances.misslab_acc[-1]
        gamma = filtered_sims[ii]["CND_reg_gamma"]
        CND_reg = filtered_sims[ii]["CND_reg_type"]
        CND_reg_only_last_layer = filtered_sims[ii]["CND_reg_only_last_layer"]

        # Apply filters
        if True: #if noise_type in selected_noise_types and CND_reg in selected_CND_regs:
            # Append a dictionary with the extracted data
            data.append({
                "name": name,
                "dataset": dataset,
                "noise_type": noise_type,
                "CND_reg": CND_reg,
                "test_acc": test_acc,
                "misslab_acc": misslab_acc,
                "gamma": gamma,
                "CND_reg_only_last_layer": CND_reg_only_last_layer
            })
    except KeyError as e:
        print(f"Missing key in performances or simulation: {e}")

# Create a DataFrame
df = pd.DataFrame(data)
df.sort_values(by=["dataset", "noise_type", "CND_reg"], inplace=True)
# Set 'name' as the index
df.set_index("name", inplace=True)

df.to_excel(os.path.join(results_dir, f"{experiment_name}.xlsx"))

# Find the best results based on test accuracy for each fixed `CND_reg`
best_results = df.loc[df.groupby(["dataset", "noise_type", "CND_reg"])["test_acc"].idxmax()]
best_results.sort_values(by=["dataset", "noise_type", "CND_reg"], inplace=True)

# Save the results as CSV using the experiment name
output_path = os.path.join(results_dir, f"{experiment_name}.csv")

best_results.to_csv(output_path)
print(f"Results saved to {output_path}")

# Optionally, display the DataFrame
ace.display_dataframe_to_user(name="Best Results for Fixed CND_reg", dataframe=best_results)

Results saved to /Volumes/T7/PhD_projects/novel_entropic_divergences_DNN_optimization/results/memorization_reg/analysis_baseline.csv
Best Results for Fixed CND_reg


In [15]:
sims = []

for folder in folders:
    if os.path.exists(f"{path}/{folder}") and os.path.isfile(f"{path}/{folder}/args.json"):
        with open(f"{path}/{folder}/args.json", 'r') as file:
            content = file.read()
            print(f"Content of {folder}/args.json:\n{content}")
            try:
                sims.append(json.loads(content))
                print(f"{folder} UPLOADED")
            except json.JSONDecodeError as e:
                print(f"Error in {folder}/args.json: {e}")
    else:
        print(f"{folder}/args.json does not exist")

Content of exp_2024-11-19_11-08/args.json:
{
    "verbose": false,
    "verbose_1": false,
    "dataset": "MNIST",
    "network": "Preact_Base_FCN",
    "n_layers": 5,
    "n_neurons_x_layer": 128,
    "batch_normalization_flag": false,
    "dropout_rate": 0.0,
    "batch_size": 512,
    "fixed_batch_size": 512,
    "epochs": 80,
    "optimizer": "SGD",
    "loss_function": "cross_entropy_loss",
    "CND_reg_type": "none",
    "CND_reg_gamma": 0.0,
    "CND_reg_only_last_layer": false,
    "metrics": [
        "CND",
        "PC"
    ],
    "PMFs_epochs_to_plot": [
        0,
        5,
        10,
        20,
        30,
        40,
        50,
        60,
        70,
        79
    ],
    "CND_type": [
        "CND_PMF",
        "CND_GL"
    ],
    "neuron_indexes": [],
    "layer_indexes": [],
    "CND_min_th": 0.0,
    "CND_min_ist": 0,
    "JSD_bins": 40,
    "noise_type": "label_noise",
    "symmetric_label_noise_ratio": 0.5,
    "lr": 0.1,
    "momentum": 0.9,
    "nesterov": tr

In [16]:
# Sort the DataFrame by 'CND_reg' and then by 'gamma'
df_sorted = df.sort_values(by=['dataset', 'noise_type', 'CND_reg', 'gamma'], ascending=[True, True, True, True])

# Optionally, display the sorted DataFrame
ace.display_dataframe_to_user(name="Sorted Results by CND_reg and gamma", dataframe=df_sorted)

results_dir

Sorted Results by CND_reg and gamma


'/Volumes/T7/PhD_projects/novel_entropic_divergences_DNN_optimization/results/memorization_reg'